# 

# Package imports

In [2]:
rm(list=ls())

In [1]:
install.packages("ncdf4")
library(ncdf4)

library(raster) # package for raster manipulation
library(ggplot2) # package for plotting
library(reshape2)

install.packages("RColorBrewer")
library(RColorBrewer)

library(tidyverse)
library(rnaturalearth)

install.packages("magick") # package for gif
library(magick)

# Download Data
The future SST
 data comes from Climate Data Store: https://cds.climate.copernicus.eu/datasets/projections-cmip6?tab=overview


# Load the data

In [4]:
fu <- nc_open("future.nc")  # read data
# Save the print(nc) to a text file
{
  sink('fu.txt')
  print(fu)
  sink()
}

In [0]:
lon <- ncvar_get(fu, "longitude")
lat <- ncvar_get(fu, "latitude", verbose = F)
time <- ncvar_get(fu, "time_bnds")


In [0]:
head(lon) # look at the first few entries in the longitude vector

In [0]:
tos.array <- ncvar_get(fu, "tos") # store the data in a 3-dimensional array
dim(tos.array) 

fillvalue <- ncatt_get(fu, "tos", "_FillValue") # view fill value
print(fillvalue)

tos.array[tos.array == fillvalue$value] <- NA # use NA to fill

# Data analysis
## Plot the Monthly average SST

In [0]:
#Loop
time_points <- 1:84

# every time points
for (time_index in time_points) {
  # Select SST data for a specific point in time
  tos_subset <- tos.array[, , time_index]
  
  # Create raster objects
  r_subset <- raster(t(tos_subset), xmn=min(lon), xmx=max(lon), ymn=min(lat), ymx=max(lat), 
                     crs=CRS("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs+towgs84=0,0,0"))
  
  # Flip the grid so that it displays correctly
  r_subset <- flip(r_subset, direction='y')
  
  # Convert raster to data frame
  r_subset_df <- as.data.frame(r_subset, xy = TRUE)
  
  # Plot
  p <- ggplot(r_subset_df, aes(x = x, y = y, fill = layer)) +  
    # Plot raster
    geom_raster() +
    # Specify fill color
    scale_fill_gradientn(colours = brewer.pal(8, "Reds"), name = "Temperature") +
    # Title
    labs(title = paste("Sea Surface Temperature - Time Point", time_index), 
         x = "Longitude", y = "Latitude") +
    # Simple theme
    theme_minimal() +
    # Add background
    theme(
      panel.background = element_rect(fill = "white"),
      plot.background = element_rect(fill = "white"),
      axis.text.x = element_text(angle = 45, hjust = 1)) +
    # Add boundaries
    geom_contour(data = r_subset_df, aes(z = layer), breaks = c(0, 25), color = "blue", size = 1) +
    geom_text(data = r_subset_df[r_subset_df$layer %in% c(0, 25), ],
              aes(label = round(layer, 1)), color = "black", vjust = -0.5)
    
  # Save plots
  ggsave(paste0("sea_surface_temperature_", time_index, ".png"), 
         plot = p, width = 6, height = 8)
}


## Make gif to show the change

In [0]:
#gif-Feb
image_files_2 <- paste0("sea_surface_temperature_", seq(2,84,by=12), ".png")  
# Read plots
img_list_2 <- image_read(image_files_2)
# Create GIF
gif_2 <- image_animate(img_list_2, fps = 1)  
# Save GIF
image_write(gif_2, "sea_surface_temperature_Feb.gif")

#gif-Jul
image_files_7 <- paste0("sea_surface_temperature_", seq(7,84,by=12), ".png")  

img_list_7 <- image_read(image_files_7)

gif_7 <- image_animate(img_list_7, fps = 1)  

image_write(gif_7, "sea_surface_temperature_Jul.gif")

## Plot the SST change for each selected point
Point 1 is located at the northern end of the humpback whale's range, Point 2 is located in the area where it begins to migrate south, and Point 3 is located in the humpback whale's range in the Antarctic region.

In [0]:
head(lon) # look at the first few entries in the longitude vector
head(lat) # look at the first few entries in the latitude vector

In [0]:
# point data
#(145.5，-15.5)
i <- 12
j <- 60

tos.n <- tos.array[i, j, ] 
print(tos.n)
tos.nf <- data.frame(
  SST = tos.n,
  year = factor(rep(seq(2030, 2090, by = 10), each = 12)))

ggplot(tos.nf, aes(x = rep(1:12, times = 7), y = SST, group = year, color = factor(year))) +
  geom_line()+
  geom_point(size = 2) +
  labs(title = "Point 1", x="month",y="SST",color="Year") +
  scale_x_continuous(breaks = 1:12) +
  theme_minimal()

#(150.5，-35.5)
i <- 17
j <- 38

tos.s <- tos.array[i, j, ] 
print(tos.s)
tos.sf <- data.frame(
  SST = tos.s,
  year = factor(rep(seq(2030, 2090, by = 10), each = 12)))

ggplot(tos.sf, aes(x = rep(1:12, times = 7), y = SST, group = year, color = factor(year))) +
  geom_line()+
  geom_point(size = 2) +
  labs(title = "Point 2", x="month",y="SST",color="Year") +
  scale_x_continuous(breaks = 1:12) +
  theme_minimal()

#(169.5，-65.5)
i <- 36
j <- 1

tos.sp <- tos.array[i, j, ] 
print(tos.sp)
tos.spf <- data.frame(
  SST = tos.sp,
  year = factor(rep(seq(2030, 2090, by = 10), each = 12)))

ggplot(tos.spf, aes(x = rep(1:12, times = 7), y = SST, group = year, color = factor(year))) +
  geom_line()+
  geom_point(size = 2) +
  labs(title = "Point 3", x="month",y="SST",color="Year") +
  scale_x_continuous(breaks = 1:12) +
  theme_minimal()


## Plot the  points' location


In [0]:
#points location
points <- data.frame(
  LON = c(145.5,150.5,169),
  LAT = c(-15.5,-35.5,-67),
  Label = c("Point 1","Point 2", "Point 3")
  )

world <- ne_countries(scale = "medium", returnclass = "sf")
ggplot() + 
  geom_sf(data = world, col="grey70", fill = "grey70")+
  geom_point(data = points, aes(x = LON, y = LAT), size = 2, shape = 21, fill = "blue", alpha=0.6) +
  geom_text(data = points, aes(x = LON, y = LAT, label = Label), vjust = -1, color = "black") +
  scale_x_continuous(expand = c(0, 0), limits = c(130, 175)) +
  scale_y_continuous(expand = c(0, 0), limits = c(-70, -10)) +
  theme_classic() + 
  theme(axis.text = element_text(colour = "black"))
